# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 9 2022 9:08AM,252703,10,Eme-106410,Emerginnova,Released
1,Dec 9 2022 9:08AM,252700,10,0086237306,ISDIN Corporation,Released
2,Dec 9 2022 9:08AM,252700,10,0086237311,ISDIN Corporation,Released
3,Dec 9 2022 8:54AM,252699,10,0086237195,ISDIN Corporation,Released
4,Dec 9 2022 8:54AM,252699,10,0086237196,ISDIN Corporation,Released
5,Dec 9 2022 8:54AM,252699,10,0086237220,ISDIN Corporation,Released
6,Dec 9 2022 8:54AM,252699,10,0086237219,ISDIN Corporation,Released
7,Dec 9 2022 8:54AM,252699,10,0086237222,ISDIN Corporation,Released
8,Dec 9 2022 8:54AM,252699,10,0086237221,ISDIN Corporation,Released
9,Dec 9 2022 8:51AM,252697,10,0086235375,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
57,252696,Released,2
58,252697,Released,5
59,252699,Released,6
60,252700,Released,2
61,252703,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
252696,NaN,NaN,2.0
252697,NaN,NaN,5.0
252699,NaN,NaN,6.0
252700,NaN,NaN,2.0
252703,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252597,0.0,0.0,1.0
252599,0.0,0.0,1.0
252600,0.0,0.0,23.0
252603,0.0,0.0,1.0
252604,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252597,0,0,1
252599,0,0,1
252600,0,0,23
252603,0,0,1
252604,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252597,0,0,1
1,252599,0,0,1
2,252600,0,0,23
3,252603,0,0,1
4,252604,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252597,,,1
1,252599,,,1
2,252600,,,23
3,252603,,,1
4,252604,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 9 2022 9:08AM,252703,10,Emerginnova
1,Dec 9 2022 9:08AM,252700,10,ISDIN Corporation
3,Dec 9 2022 8:54AM,252699,10,ISDIN Corporation
9,Dec 9 2022 8:51AM,252697,10,ISDIN Corporation
14,Dec 9 2022 8:49AM,252694,12,Hush Hush
15,Dec 9 2022 8:47AM,252696,19,"GCH Granules USA, Inc."
17,Dec 9 2022 8:39AM,252693,19,ACG North America LLC
18,Dec 9 2022 8:17AM,252691,19,"GUSA Granules USA, Inc."
19,Dec 9 2022 7:40AM,252690,19,"AdvaGen Pharma, Ltd"
25,Dec 8 2022 5:06PM,252687,102,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 9 2022 9:08AM,252703,10,Emerginnova,,,1
1,Dec 9 2022 9:08AM,252700,10,ISDIN Corporation,,,2
2,Dec 9 2022 8:54AM,252699,10,ISDIN Corporation,,,6
3,Dec 9 2022 8:51AM,252697,10,ISDIN Corporation,,,5
4,Dec 9 2022 8:49AM,252694,12,Hush Hush,,,1
5,Dec 9 2022 8:47AM,252696,19,"GCH Granules USA, Inc.",,,2
6,Dec 9 2022 8:39AM,252693,19,ACG North America LLC,,,1
7,Dec 9 2022 8:17AM,252691,19,"GUSA Granules USA, Inc.",,,1
8,Dec 9 2022 7:40AM,252690,19,"AdvaGen Pharma, Ltd",,6,
9,Dec 8 2022 5:06PM,252687,102,"Senseonics, Incorporated",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 9:08AM,252703,10,Emerginnova,1,,
1,Dec 9 2022 9:08AM,252700,10,ISDIN Corporation,2,,
2,Dec 9 2022 8:54AM,252699,10,ISDIN Corporation,6,,
3,Dec 9 2022 8:51AM,252697,10,ISDIN Corporation,5,,
4,Dec 9 2022 8:49AM,252694,12,Hush Hush,1,,
5,Dec 9 2022 8:47AM,252696,19,"GCH Granules USA, Inc.",2,,
6,Dec 9 2022 8:39AM,252693,19,ACG North America LLC,1,,
7,Dec 9 2022 8:17AM,252691,19,"GUSA Granules USA, Inc.",1,,
8,Dec 9 2022 7:40AM,252690,19,"AdvaGen Pharma, Ltd",,6,
9,Dec 8 2022 5:06PM,252687,102,"Senseonics, Incorporated",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 9:08AM,252703,10,Emerginnova,1,,
1,Dec 9 2022 9:08AM,252700,10,ISDIN Corporation,2,,
2,Dec 9 2022 8:54AM,252699,10,ISDIN Corporation,6,,
3,Dec 9 2022 8:51AM,252697,10,ISDIN Corporation,5,,
4,Dec 9 2022 8:49AM,252694,12,Hush Hush,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 9:08AM,252703,10,Emerginnova,1.0,NaN,NaN
1,Dec 9 2022 9:08AM,252700,10,ISDIN Corporation,2.0,NaN,NaN
2,Dec 9 2022 8:54AM,252699,10,ISDIN Corporation,6.0,NaN,NaN
3,Dec 9 2022 8:51AM,252697,10,ISDIN Corporation,5.0,NaN,NaN
4,Dec 9 2022 8:49AM,252694,12,Hush Hush,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 9:08AM,252703,10,Emerginnova,1.0,0.0,0.0
1,Dec 9 2022 9:08AM,252700,10,ISDIN Corporation,2.0,0.0,0.0
2,Dec 9 2022 8:54AM,252699,10,ISDIN Corporation,6.0,0.0,0.0
3,Dec 9 2022 8:51AM,252697,10,ISDIN Corporation,5.0,0.0,0.0
4,Dec 9 2022 8:49AM,252694,12,Hush Hush,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5053113,141.0,3.0,7.0
102,252687,1.0,0.0,0.0
12,252694,1.0,0.0,0.0
15,505371,7.0,0.0,0.0
19,2526738,9.0,11.0,1.0
20,505250,2.0,0.0,0.0
21,5558080,22.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5053113,141.0,3.0,7.0
1,102,252687,1.0,0.0,0.0
2,12,252694,1.0,0.0,0.0
3,15,505371,7.0,0.0,0.0
4,19,2526738,9.0,11.0,1.0
5,20,505250,2.0,0.0,0.0
6,21,5558080,22.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,141.0,3.0,7.0
1,102,1.0,0.0,0.0
2,12,1.0,0.0,0.0
3,15,7.0,0.0,0.0
4,19,9.0,11.0,1.0
5,20,2.0,0.0,0.0
6,21,22.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,141.0
1,102,Released,1.0
2,12,Released,1.0
3,15,Released,7.0
4,19,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,19,20,21
Status,,,,,,,
Completed,7.0,0.0,0.0,0.0,1.0,0.0,0.0
Executing,3.0,0.0,0.0,0.0,11.0,0.0,0.0
Released,141.0,1.0,1.0,7.0,9.0,2.0,22.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,19,20,21
0,Completed,7.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,3.0,0.0,0.0,0.0,11.0,0.0,0.0
2,Released,141.0,1.0,1.0,7.0,9.0,2.0,22.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,19,20,21
0,Completed,7.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,3.0,0.0,0.0,0.0,11.0,0.0,0.0
2,Released,141.0,1.0,1.0,7.0,9.0,2.0,22.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()